In [119]:
from sympy import *
import sympy
from sympy.abc import *

In [120]:
t=Symbol('t', positive=True, integer=True)
w=Function('w')
a,b,c=symbols('a b c')
S=Function('S')
k=Symbol('k')
P=Function('P')
W=Function('W')

eq1=Eq(diff(w(t), t), a*w(t)*(1-w(t)/b)-c*S(t)*w(t))
eq1

Eq(Derivative(w(t), t), a*(1 - w(t)/b)*w(t) - c*S(t)*w(t))

In [135]:
ICmax=Function('IC_{max}')
LAI=Function('LAI')
eq5=Eq(
ICmax(t), (0.935)+ LAI(t)*(0.498) - (LAI(t)**2)*(0.00575)
)
eq5

Eq(IC_{max}(t), -0.00575*LAI(t)**2 + 0.498*LAI(t) + 0.935)

In [132]:
cp=Function('cp')
eq3=Eq(
    cp(t), ((1-exp(-eg*w(t)/w_star)) * 
   (alpha_star - W(t)* (1-exp(eg*w(t)/w_star))-1.8*S(t)))
)
eq3

Eq(cp(t), (1 - exp(-\epsilon_g*w(t)/w^*))*(\alpha^* - (1 - exp(\epsilon_g*w(t)/w^*))*W(t) - 1.8*S(t)))

In [154]:
cp=((1-exp(-eg*w(t)/w_star)) * 
   (alpha_star - W(t)* (1-exp(eg*w(t)/w_star))-1.8*S(t)))
cp

(1 - exp(-\epsilon_g*w(t)/w^*))*(\alpha^* - (1 - exp(\epsilon_g*w(t)/w^*))*W(t) - 1.8*S(t))

In [138]:
ICstore=Function('ICstore')

eq4=Eq(
    ICstore(t), cp(t) *ICmax(t) *(1-exp(-k*P(t)/ICmax(t)))
)
eq4

Eq(ICstore(t), (1 - exp(-k*P(t)/IC_{max}(t)))*IC_{max}(t)*cp(t))

In [155]:
ICstore=cp *ICmax(t) *(1-exp(-k*P(t)/ICmax(t)))
ICstore

(1 - exp(-\epsilon_g*w(t)/w^*))*(1 - exp(-k*P(t)/IC_{max}(t)))*(\alpha^* - (1 - exp(\epsilon_g*w(t)/w^*))*W(t) - 1.8*S(t))*IC_{max}(t)

In [139]:
alpha_star=Symbol(r'\alpha^*')
eg=Symbol(r'\epsilon_g')
w_star=Symbol(r'w^*')

In [156]:
E=Function('E')
eq2=Eq(W(t).diff(),
  P(t)-E(t)-ICstore )
eq2

Eq(Derivative(W(t), t), -(1 - exp(-\epsilon_g*w(t)/w^*))*(1 - exp(-k*P(t)/IC_{max}(t)))*(\alpha^* - (1 - exp(\epsilon_g*w(t)/w^*))*W(t) - 1.8*S(t))*IC_{max}(t) - E(t) + P(t))

In [129]:
dsolve(eq1, w(t))

Eq(w(t), b*exp(a*t - c*Integral(S(t), t))/(C1*b + a*Integral(exp(a*t)*exp(-c*Integral(S(t), t)), t)))

In [158]:
dsolve([eq1, eq2], [w(t),W(t)])

NotImplementedError: 